# Portfolio Experiment

In [3]:
library('dplyr')
library('ggplot2')
library('abind')
library('deSolve')
library('matrixcalc')
library('mbend')
library('readr')

In [ ]:
source('PortfolioFunctions_v7.R')

##  Reading datasets

In [ ]:
# Reading datasets
down <- read.csv('df_down_1999.csv') %>%
  mutate(Date=as.Date(Date, format = "%Y-%m-%d")) %>% 
  filter(Date >= '2000-01-01')

ftse100 <- read.csv('df_FTSE_1999.csv')%>%
  mutate(Date=as.Date(Date, format = "%Y-%m-%d")) %>% 
  filter(Date >= '2000-01-01')

dax <- read.csv('df_DAX_1999.csv')%>%
  mutate(Date=as.Date(Date, format = "%Y-%m-%d")) %>% 
  filter(Date >= '2000-01-01') 


sp500 <- read.csv('df_S&P500_1999.csv')%>%
  mutate(Date=as.Date(Date, format = "%Y-%m-%d")) %>% 
  filter(Date >= '2000-01-01')  

nasdaq <- read.csv('df_NASDAQ_1999.csv')%>%
  mutate(Date=as.Date(Date, format = "%d/%m/%Y")) %>% 
  filter(Date >= '2000-01-01') %>% 
  select(-HSKA)

nasdaqETF <- read.csv('df_NASDAQ.ETF_1999.csv')%>%
  mutate(Date=as.Date(Date, format = "%d/%m/%Y")) %>% 
  filter(Date >= '2000-01-01') 

nyse <- read.csv('df_NYSE_1999.csv')%>%
  mutate(Date=as.Date(Date, format = "%d/%m/%Y")) %>% 
  filter(Date >= '2000-01-01') %>% select(-AIG, -BAC, -AEG, -BIG, -BDN, -BBVA, -AWR, -AME, -AFL)
colnames(nasdaqETF)


In [ ]:
################################################################################
######   This code generarte the dataset for multiple cases listed below   #####
################################################################################
# iterates the first k stocks: from 6 to 10
top_vect <- 6:10  # stocks selected
# iterates the period of rebalance (in months)
period <- c(1)  # period 1 month
# iterates the type of sort from k stocks
sort <- c('mean')  # method of selecte stocks
# iterates the type of covariance matrix
matrix <- c('mv', 'lm')  # model
# iterates the list od datasets
df_completed <- list('down' = down,
                     'ftse100' = ftse100, 
                     'dax'= dax,
                     'sp500' = sp500, 
                     'nyse'= nyse,
                     'nasdaq' = nasdaq,
                     'nasdaqETF' = nasdaqETF)
# iterates the respective year for each dataset
years_start <- c('2007', '2007', '2007', '2007', '2007', '2007', '2007')
# initial datasets
base.portfolio.ret <- NULL
base.portfolio.cum.ret <- NULL
cv_mv <- list()
cv_exp <- list()
base.re_sd_port <- NULL
# loop for all the iterators
for(j in 1:length(df_completed)){
  for(k in 1:length(period)){
    for(l in 1:length(sort)){
      for(i in 1:length(top_vect)){
          # mean variance portfolio  
          name <- paste(names(df_completed)[j], top_vect[i], sep='_')
          port.mv <- getPortfolio(base = df_completed[[j]], 
                                  year_to_start = years_start[j], 
                                  rebalance_period = period[k], 
                                  mod = 'mv', 
                                  sorted= sort[l] , 
                                  top.k = top_vect[i])
          # covariances
          cv_mv[[name]] <- port.mv$matCov
          # lm portfolio
          port.lm <- getPortfolio(base = df_completed[[j]], 
                                  year_to_start = years_start[j], 
                                  rebalance_period = period[k], 
                                  mod = 'lm', 
                                  sorted= sort[l] , 
                                  top.k = top_vect[i])
          # covariances
          cv_exp[[name]] <- port.lm$matCov
          #cat('Dim of CV_exp: ', dim(cv_exp),'\n')
          # Portfolio returns
          names <- c('date', 'sorted_by', 'rebalance_period', 'base', 'top', 'MV', 'MV-LM', 'MV-EW')
          portfolio.ret <- data.frame('date' = port.mv$df.port.ret$date,
                                      'sorted_by' = sort[l],
                                      'rebalance_period' = period[k],
                                      'base' = names(df_completed)[j],
                                      'top' = top_vect[i],
                                      #'mod' = ,
                                      'MV'= port.mv$df.port.ret$min.ret,
                                      'MV.LM'= port.lm$df.port.ret$min.ret,
                                      'MV.EW' = port.mv$df.port.ret$eqw.ret)
          # portfolio expected return
          portfolio.re_sd <- data.frame('date' = port.mv$df_re_sd.port$date,
                                        'sorted_by' = sort[l],
                                        'rebalance_period' = period[k],
                                        'base' = names(df_completed)[j],
                                        'top' = top_vect[i],
                                        're.p_mv' = port.mv$df_re_sd.port$re.port,
                                        'sd.p_mv' = port.mv$df_re_sd.port$sd.port,
                                        're.p_lm' = port.lm$df_re_sd.port$re.port,
                                        'sd.p_mv' = port.lm$df_re_sd.port$sd.port)
          print(portfolio.re_sd)
          # Cummulative returns
          port.cum.ret <- portfolio.ret %>%
            mutate(cum.MV = cumsum(MV),
                   cum.MV.LM = cumsum(MV.LM),
                   cum.MV.EW = cumsum(MV.EW)) %>%
            dplyr::select(-MV, -MV.LM, -MV.EW)
          ## colnames
          colnames(portfolio.ret) <- names
          colnames(port.cum.ret) <- names 
          # join tables
          base.portfolio.ret <- rbind(base.portfolio.ret, portfolio.ret)
          base.portfolio.cum.ret <- rbind(base.portfolio.cum.ret, port.cum.ret)
          base.re_sd_port <- rbind(base.re_sd_port, portfolio.re_sd)
      } 
    }
  }
}

In [ ]:
## write results
write_rds(base.re_sd_port, './base.portfolio.re_and_sd_2000.rds')
write_rds(base.portfolio.ret, './base.portfolio.ret_2000_complete.rds')
write_rds(base.portfolio.cum.ret, './base.portfolio.cum.ret_2000_complete.rds')
